# Introduction to Probabilistic Programming

## Introduction



## Preliminaries

### Probability

In [97]:
type Probability = private Probability of double

module Probability =
    let create ( probability : double ) : Probability option =
        if probability >= 0.0 && probability <= 1.0 then Some ( Probability probability )
        else None

    let getProbability ( Probability probability ) = probability

#### Example

In [101]:
// Creating
let prob = Probability.create ( 1.0/2.0 )

// Getting the value
match prob with
    | Some p -> Probability.getProbability p
    | None   -> nan

0.5

### Distribution

In [102]:
type Distribution = seq< int * Probability >

#### Uniform Distribution

In [104]:
let uniform ( x : seq< int > ) : Distribution  = 
    seq {
        let countOfSequence = Seq.length x
        let distributionSequence =
            x 
            |> Seq.map( fun e -> e, Probability.create ( 1.0 / double( countOfSequence )))
            |> Seq.filter( fun ( e, p ) -> p.IsSome )
            |> Seq.map( fun (e, p) -> e, p.Value )
        yield! distributionSequence
    }

In [106]:
uniform [ 0..1 ]

seq [(0, Probability 0.5); (1, Probability 0.5)]

## The Monty Hall Problem

## Bayesian Inference

### A/B Test

In [60]:
let posteriorDistribution ( data : double                ) 
                          ( priorSampler : seq<double>   ) 
                          ( simulator : double -> double ) : seq<double option> = 
    seq {
        for p in priorSampler do
            printfn "Simulator: %A" ( simulator p )
            printfn "p: %A" ( p )
            if simulator( p ) = data then yield Some p
            else yield None
    }

In [61]:
open System

let seed   = 123 
let random = Random( seed )

let simulateConversion ( p : double ) ( nVisitors : int ) : double = 
    [ 1..nVisitors ]
    |> List.map( fun x -> random.NextDouble() )
    |> List.filter( fun d -> d < p )
    |> List.sum

// Print out the Simulated Conversions.
printfn "%f" ( simulateConversion 0.1 1000 )
printfn "%f" ( simulateConversion 0.1 1000 )
printfn "%f" ( simulateConversion 0.1 1000 )

5.380237
5.741275
4.741965


In [66]:
open System

// [ 0, 1 ]
let uniformPriorSampler =
    seq { while true do yield random.NextDouble() } 
    
Seq.take 3 uniformPriorSampler

seq [0.06489795682; 0.6294309779; 0.2492824831]

In [63]:
let applySimulation ( p : double ) : double  =
    simulateConversion p 100
    
let posterior = 
    posteriorDistribution 0.4 uniformPriorSampler applySimulation
let r = 
    posterior
    |> Seq.take 1000
    |> Seq.filter( fun x -> x.IsSome )
    |> Seq.toList

printfn "%A" r

Simulator: 0.6037161646
p: 0.154412129
Simulator: 10.29143786
p: 0.4015697517
Simulator: 3.024213957
p: 0.2676955928
Simulator: 1.23474038
p: 0.169650754
Simulator: 0.7609393223
p: 0.1415839638
Simulator: 10.09192572
p: 0.5003337779
Simulator: 6.806743869
p: 0.3997930192
Simulator: 8.655780263
p: 0.4036144965
Simulator: 2.610437434
p: 0.2109703963
Simulator: 29.30636305
p: 0.700215713
Simulator: 13.17359426
p: 0.5410964841
Simulator: 23.39536809
p: 0.6701997056
Simulator: 1.409745427
p: 0.1490956667
Simulator: 29.27799402
p: 0.7491804942
Simulator: 2.368824086
p: 0.2242644197
Simulator: 0.09115779497
p: 0.06037117357
Simulator: 9.018457155
p: 0.4325975275
Simulator: 48.85948451
p: 0.9475070149
Simulator: 32.05578239
p: 0.7816777694
Simulator: 0.6003989743
p: 0.1183922464
Simulator: 5.367501034
p: 0.2927190905
Simulator: 38.42572681
p: 0.8325641955
Simulator: 46.38205082
p: 0.9333129711
Simulator: 39.56184103
p: 0.9449696457
Simulator: 3.716192336
p: 0.2157293922
Simulator: 7.158433655
